# Eigenmode simulation with Ports

Authors: Samarth Hawaldar, Arvind Mamgain

Adapted from tutorial 4.16


In [ ]:
# Import useful packages
import qiskit_metal as metal
from qiskit_metal import designs, draw
from qiskit_metal import MetalGUI, Dict, open_docs

# Packages for the simple design
from qiskit_metal.qlibrary.tlines.meandered import RouteMeander
from qiskit_metal.qlibrary.tlines.pathfinder import RoutePathfinder
from qiskit_metal.qlibrary.terminations.launchpad_wb_driven import LaunchpadWirebondDriven
from qiskit_metal.qlibrary.terminations.open_to_ground import OpenToGround
from qiskit_metal.qlibrary.couplers.coupled_line_tee import CoupledLineTee
# Analysis
from qiskit_metal.analyses.quantization import EPRanalysis

# Set up the design

In [ ]:
# Set up chip dimensions 
design = designs.DesignPlanar()
design._chips['main']['size']['size_x'] = '2mm'
design._chips['main']['size']['size_y'] = '2mm'
design._chips['main']['size']['size_z'] = '-280um'
# Resonator and feedline gap width (W) and center conductor width (S) from reference 2
design.variables['cpw_width'] = '10 um' #S from reference 2
design.variables['cpw_gap'] = '6 um' #W from reference 2


design.overwrite_enabled = True

hfss = design.renderers.hfss

# Open GUI
gui = MetalGUI(design)

# Define the geometry

Here we will have a single feedline couple to a single CPW resonator.

The lauchpad should be included in the driven model simulations.

For that reason, we use the LaunchpadWirebondDriven component which has an extra pin for input/output

In [ ]:
###################
# Single feedline #
###################

# Driven Lauchpad 1
x = '-0.5mm'
y = '0.5mm'
launch_options = dict(chip='main', pos_x=x, pos_y=y, orientation='360', lead_length='30um')
LP1 = LaunchpadWirebondDriven(design, 'LP1', options = launch_options)

# Driven Launchpad 2
x = '0.5mm'
y = '0.5mm'
launch_options = dict(chip='main', pos_x=x, pos_y=y, orientation='180', lead_length='30um')
LP2 = LaunchpadWirebondDriven(design, 'LP2', options = launch_options)

# coupling resonator to feedline 
q_read = CoupledLineTee(design,'Q_Read_T', options=dict(pos_x = '0.0mm', pos_y = '0.5mm', 
                                                        orientation = '0',
                                                        coupling_space = '6um',                                                         
                                                        coupling_length = '300um',
                                                        open_termination = False))


gui.rebuild()


In [ ]:

# Using path finder to connect the two launchpads
TL_LP1_T = RoutePathfinder(design, 'TL_LP1_T', options = dict(chip='main', trace_width ='10um',
        trace_gap ='6um',
                                            fillet='99um',
                                                                  
        hfss_wire_bonds = True,
                                            lead=dict(end_straight='0.1mm'),
                                            pin_inputs=Dict(
                                                start_pin=Dict(
                                                    component='LP1',
                                                    pin='tie'),
                                                end_pin=Dict(
                                                    component='Q_Read_T',
                                                    pin='prime_start')
                                            )))

TL_T_LP2 = RoutePathfinder(design, 'TL_T_LP2', options = dict(chip='main', trace_width ='10um',
        trace_gap ='6um',
                                            fillet='99um',
                                                                  
        hfss_wire_bonds = True,
                                            lead=dict(end_straight='0.1mm'),
                                            pin_inputs=Dict(
                                                start_pin=Dict(
                                                    component='Q_Read_T',
                                                    pin='prime_end'),
                                                end_pin=Dict(
                                                    component='LP2',
                                                    pin='tie')
                                            )))
# # Rebuild the GUI

In [ ]:
######################
# lambda/4 resonator #
######################

# First we define the two end-points
otg = OpenToGround(design, 'otg', options=dict(chip='main', pos_x='0.0mm',  pos_y='-0.7mm', orientation='-90'))

# Use RouteMeander to fix the total length of the resonator
rt_meander = RouteMeander(design, 'meander',  Dict(
        trace_width ='10um',
        trace_gap ='6um',
        total_length='3.7mm',
        hfss_wire_bonds = True,
        fillet='99 um',
        lead = dict(start_straight='250um'),
        pin_inputs=Dict(
            start_pin=Dict(component='otg', pin='open'),
            end_pin=Dict(component='Q_Read_T', pin='second_end')), ))

# rebuild the GUI
gui.rebuild()

In [ ]:
gui.autoscale()
gui.screenshot()

# Perfoming Eigenmode Simulation

In [ ]:
from qiskit_metal.analyses.quantization import EPRanalysis
eig_qb = EPRanalysis(design, "hfss")
hfss = eig_qb.sim.renderer
hfss.start()

In [ ]:
em_p = eig_qb.sim.setup
em_p.name = 'HangerReso'
em_p.min_freq_ghz = 1
em_p.n_modes = 1
em_p.max_passes = 15
em_p.max_delta_f = 0.1
em_p.min_converged = 2
em_p.basis_order = 2
# Design variables can also be added in for direct simulation sweeps.
em_p.vars = Dict({'Lj': '0nH', 'Cj': '0 fF'})

In [ ]:
eig_qb.sim._render(name="HangerReso",
                selection = [],
                solution_type='eigenmode',
                vars_to_initialize=em_p.vars, 
                open_pins=[], 
                port_list=[('LP1', 'in','50'), ('LP2', 'in','50')],
                box_plus_buffer = False)

In [ ]:
eig_qb.sim._analyze()

In [ ]:
eig_qb.sim.plot_convergences()

In [ ]:
#eig_qb.sim.save_screenshot()       #optional

In [ ]:
eig_qb.get_frequencies()

In [ ]:
hfss.close()

In [ ]:
# Data obtained during testing
# Data ANSYS:
# Mesh Size = 50um, Q = 18875, f = 7.3167
# Mesh Size = 15um, Q = 4269, f = 7.3031
# Mesh Size = 7um, Q = 14943, f = 7.3316
# Data COMSOL:
# Mesh = Normal, Q = 18858, f = 7.1315
# Mesh = Fine, Q = 18584, f = 7.1176
# Mesh = Finer, Q = 17042, f = 7.1426
# Mesh = Extra Fine, Q = 16986, f = 7.2079
# Mesh = Extremely Fine, Q = 14591, f = 7.2092